In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy  as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import regularizers
from keras import optimizers
from ann_visualizer.visualize import ann_viz
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
%matplotlib inline
from catboost import CatBoostClassifier, FeaturesData
import datetime as DT
import io
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN 
from collections import Counter
from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


Importando arquivos

In [3]:
tabelao = pd.read_csv('tabelao_idade.csv')

In [4]:
tabelao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96406 entries, 0 to 96405
Data columns (total 21 columns):
CODIGO_CLIENTE             96406 non-null int64
DATA_NASCIMENTO            96403 non-null object
IDADE                      96406 non-null int64
SEXO                       96405 non-null object
ESTADO_CIVIL               96405 non-null object
CIDADE                     96406 non-null object
UF                         96406 non-null object
BAIRRO                     96406 non-null object
CEP                        96406 non-null int64
QTD_DEPENDENTES            96405 non-null float64
DATA_CADASTRO              96406 non-null object
DATA_ADMISSAO              52050 non-null object
CATEGORIAL_PROFISSAO       94690 non-null object
TIPO_RESIDENCIA            95454 non-null object
RENDA_TITULAR              96400 non-null float64
RENDA_CJ                   4393 non-null float64
OUTRAS_RENDAS              11070 non-null float64
QTD_CARTOES_ADICIONAIS     96406 non-null int64
FLAG_CONTA

In [5]:
tabelao = tabelao.drop(columns=['CODIGO_CLIENTE', 'QTD_DEPENDENTES','DATA_NASCIMENTO','CIDADE','UF', 'BAIRRO','CEP', 'DATA_CADASTRO','DATA_ADMISSAO',
             'RENDA_CJ','OUTRAS_RENDAS','FLAG_CONTA_BANCO',
                                'QTD_CARTOES_ADICIONAIS','VALOR_HISTORICO_COMPRAS', 'SEXO'], axis=1)

In [6]:
tabelao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96406 entries, 0 to 96405
Data columns (total 6 columns):
IDADE                   96406 non-null int64
ESTADO_CIVIL            96405 non-null object
CATEGORIAL_PROFISSAO    94690 non-null object
TIPO_RESIDENCIA         95454 non-null object
RENDA_TITULAR           96400 non-null float64
CLASS                   96406 non-null int64
dtypes: float64(1), int64(2), object(3)
memory usage: 4.4+ MB


In [7]:
tabelao.head()

,IDADE,ESTADO_CIVIL,CATEGORIAL_PROFISSAO,TIPO_RESIDENCIA,RENDA_TITULAR,CLASS
0,63,SO,APOSENTADO,PR,2300.0,1
1,51,CA,ASSALARIADO,OU,1500.0,1
2,30,SO,AUT+NOMO,PR,1000.0,1
3,65,SO,APOSENTADO,OU,1050.0,1
4,62,SO,AUT+NOMO,PR,1200.0,1


In [8]:
tabelao.CLASS.value_counts()

1    72127
0    24279
Name: CLASS, dtype: int64

## TABELA DE MODELAGEM

In [9]:
df_1 = tabelao[tabelao.CLASS==1].iloc[:10000,:]
df_0 = tabelao[tabelao.CLASS==0].iloc[:10000,:]

In [10]:
df_model = df_1.append(df_0)

In [11]:
df_2 = tabelao[tabelao.CLASS==1].iloc[10000:,:]
df_3 = tabelao[tabelao.CLASS==0].iloc[10000:,:]

In [12]:
df_score = df_2.append(df_3)

In [13]:
# df_model = pd.get_dummies(df_model, columns=['ESTADO_CIVIL', 'CATEGORIAL_PROFISSAO',
#                                            'TIPO_RESIDENCIA'], drop_first=True)

In [14]:
df_model = df_model.dropna()

In [15]:
# # Verificar qual a coluna 'CLASS'
# X = df_model.iloc[:, :].values
# X = np.delete(X, obj= 2, axis= 1)
# y = df_model.iloc[:, 2].values

In [16]:
modelling = pd.read_csv('df_model.csv')

In [17]:
modelling.head()

,IDADE,RENDA_TITULAR,ESTADO_CIVIL_CO,ESTADO_CIVIL_SE,ESTADO_CIVIL_SO,ESTADO_CIVIL_VI,CATEGORIAL_PROFISSAO_APOSENTADO,CATEGORIAL_PROFISSAO_ASSALARIADO,CATEGORIAL_PROFISSAO_AUT+NOMO,CATEGORIAL_PROFISSAO_LIBERAL,TIPO_RESIDENCIA_FA,TIPO_RESIDENCIA_FI,TIPO_RESIDENCIA_OU,TIPO_RESIDENCIA_PR,TIPO_RESIDENCIA_SM,CLASS
0,63,2300.0,0,0,1,0,1,0,0,0,0,0,0,1,0,1
1,51,1500.0,0,0,0,0,0,1,0,0,0,0,1,0,0,1
2,30,1000.0,0,0,1,0,0,0,1,0,0,0,0,1,0,1
3,65,1050.0,0,0,1,0,1,0,0,0,0,0,1,0,0,1
4,62,1200.0,0,0,1,0,0,0,1,0,0,0,0,1,0,1


In [18]:
X= modelling.iloc[:,:-1].values
y = modelling.iloc[:,-1].values

In [19]:
unique_elements, counts_elements = np.unique(y, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[   0    1]
 [9999 9999]]


In [20]:
skf = StratifiedKFold(n_splits=4,)

In [23]:
#XBOOST
seed = 7
test_size = 0.25

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

#scaling
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)


model1 = xgb.XGBClassifier()
model2 = xgb.XGBClassifier(n_estimators=100, max_depth=8, learning_rate=0.1, subsample=0.5)

train_model1 = model1.fit(X_train, y_train)
pred1 = train_model1.predict(X_test)

print('Model 1 XGboost Report',classification_report(y_test, pred1))
print("Accuracy for model 1: %.2f" % (accuracy_score(y_test, pred1) * 100))

Model 1 XGboost Report               precision    recall  f1-score   support

           0       0.60      0.51      0.55      2572
           1       0.55      0.64      0.59      2428

   micro avg       0.57      0.57      0.57      5000
   macro avg       0.58      0.57      0.57      5000
weighted avg       0.58      0.57      0.57      5000

Accuracy for model 1: 57.24


KERAS COM SMOTEEN

In [ ]:
# # RNA

# for train_index, test_index in skf.split(X, y):
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]
    
#     #scaling
#     sc_X = StandardScaler()
#     X_train = sc_X.fit_transform(X_train)
#     X_test = sc_X.transform(X_test)
    
# #     #combination over and under sampling
# #     sme = SMOTEENN()
# #     X_res, y_res = sme.fit_resample(X_train, y_train)
    
#     #KERAS
#     model_1 = Sequential()

#     # Input e 1ª camada oculta
#     model_1.add(Dense(15,
#                     input_shape=(15,),
#                     kernel_initializer = 'uniform',
#                     activation = 'relu'))

    
#     # 2ª e 3ª camada oculta
#     model_1.add(Dense(8,
#                       kernel_initializer = 'uniform',
#                       activation = 'relu'))
    
#     # output
#     model_1.add(Dense(1, kernel_initializer = 'uniform', activation = 'sigmoid'))

#     # Compile model

#     model_1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#     history_1 = model_1.fit(X_train, y_train, epochs=100, validation_split=0.2)
    
#     #classification report
#     y_pred_prob_1 = model_1.predict(X_test)
#     y_pred_1 = (y_pred_prob_1 > 0.5)
    
#     print('keras1 report:', classification_report(y_test, y_pred_1))

In [ ]:
# print('AUC_1: %.3f' % roc_auc_score(y_test, y_pred_prob_1))


# print("Accuracy for keras1: %.2f" % (accuracy_score(y_test, y_pred_1) * 100))

In [ ]:
# # Historico da Acurácia
# plt.plot(history_1.history['acc'], color='black')
# plt.plot(history_1.history['val_acc'], color='red')
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

# # Histórico do erro
# plt.plot(history_1.history['loss'], color='black')
# plt.plot(history_1.history['val_loss'], color='red')
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()